<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [5]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import ee
import geemap
import os

In [7]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Download an ee.Image

In [9]:
image = ee.Image('COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')

sentinel_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(image,sentinel_vis, "SENTINEL 2A", True, 0.7)

In [11]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                
              [102.8139724155494,2.4971168113761966],
              [102.82066720925057,2.4948873306759434],
              [102.82959360085214,2.4971168113761966],
              [102.8275336643287,2.5027762453853604],
              [102.81568902931893,2.5027762453853604],
              [102.8139724155494,2.4971168113761966],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

In [24]:
out_dir = os.path.join(os.path.expanduser('c:/mockdownload'), 'downloads')
filenamexp = os.path.join(out_dir, 'sentinel2.tif')

In [20]:
print(out_dir)

c:/mockdownload/downloads


### Exporting all bands as one single image

In [25]:
image = image.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filenamexp, scale=90, region=roi, file_per_band=False
)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: ignored

### Exporting each band as one image

In [26]:
geemap.ee_export_image(
    image, filename=filenamexp, scale=90, region=roi, file_per_band=True
)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: ignored

### Export an image to Google Drive

In [27]:
geemap.ee_export_image_to_drive(
    image, description='sentinel2', folder='ILSM', region=roi, scale=30
)

## Download an ee.ImageCollection

In [ ]:
import ee
import geemap
import os

In [ ]:
loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(loc)
    .filterDate('2008-01-01', '2020-01-01')
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [ ]:
print(collection.aggregate_array('system:index').getInfo())

In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)

In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

## Extract pixels as a Numpy array

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810').select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
    [[[-110.8, 44.7], [-110.8, 44.6], [-110.6, 44.6], [-110.6, 44.7]]], None, False
)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image.
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255 * ((rgb_img[:, :, 0:3] - 100) / 3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()